# Gradient Boosting Machines (GBM)

### Adboosting

* **Zayıf tahmin eden yapıları(öğrenileri) bir araya getirerek güçlü öğreniciler yaratmak mantığına dayanır.**

### GBM
* **Agboosting yapılarına kolayca uyarlanabilen genelleştirişmiş versiyondur.**
* **Hatalar üzerine kurulan tek bir tahminsel modeller serili oluşturulur.**
* **Seri içerisindeki bir model bir önceki modelin hatalarını/artıklarını(residuals) kullanarak kurulur(fit).**
* **GBM bir çok temel öğrenici(base learner type) tipi kullanılabilir.(Tree, linear models, splines..)**
* **GBM'de en önemlisi diferansiyellenebilen kayıp fonksiyonları optimize edilmesidir.**
* **Boosting + Gradient Descent = GBM**

### Model & tahmin

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor  #Random forest için
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
#veri
df = pd.read_csv("beyzbol_veri.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
#model
gbm_model = GradientBoostingRegressor().fit(X_train, y_train)
y_pred = gbm_model.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE
gbm_model

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Model Tuning

* **HiperParametreler:**
* **criterion : Bölünmeler ile ilgili saflık ölçüsünü ifade etmektedir.**
* **learnin_rate : Ağaçların katkısı ile ilgili bir değerdir.**
* **loss : en küçük hataları seçebilen yöntemler.**
* **max_features : Gözönünde bulundurulacak değişken sayısı.**
* **n_estimators : Ağaç sayısını ifade etmekte**
* **subsample : Oluşturulacak olan ağaçları oluşturulurken göz önünde bulunduracak oranı ifade etmekte.**

In [9]:
gbm_model = GradientBoostingRegressor().fit(X_train, y_train)

In [10]:
params = {"learning_rate": [0.1, 0.01],
         "max_depth": [3 ,10],
         "n_estimators": [500, 2000],
         "subsample": [1, 0.5]}

In [11]:
gbm_cv_model = GridSearchCV(gbm_model, params, cv = 10, n_jobs = -1, verbose = 2).fit(X_train, y_train)

Fitting 10 folds for each of 16 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.5min finished


In [12]:
gbm_cv_model.best_params_

{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500, 'subsample': 0.5}

In [13]:
gbm_tuned = GradientBoostingRegressor(learning_rate = 0.1,
                                     max_depth = 10,
                                     n_estimators = 500,
                                     subsample = 0.5).fit(X_train, y_train)

In [14]:
y_pred = gbm_tuned.predict(X_test)

In [15]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE

332.78359455662536